# GANの実装
- Google Colabratory上での実装を想定している点にご注意ください 

## Googleドライブをマウント（生成画像をドライブに保存するため）

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
!ls drive/My\ Drive/GAN_images

## 必要なライブラリのインポート

In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# GANの基本構造の構築
- 入力画像の型定義
- Discriminatorのモデル定義（それぞれの層、活性化関数など）、コンパイル
- Generatorのモデル定義（それぞれの層、活性化関数など）、コンパイル（コンビネーションさせること注意）

参考
- Gneratorの活性化関数LeakyRELUについては[こちら](http://www.thothchildren.com/chapter/59b93f7575704408bd4300f2)　　
- 最適化手法の種類については[こちら](https://qiita.com/tkazusa/items/4562cc7080105d5c78a9)

In [2]:
#Gモデルの構築関数
def build_generator():
    noise_shape = (z_dim,)
    model = Sequential()

    #入力層（入力の型はランダムノイズz、出力の型は256ノード、活性化関数はReLU）
    model.add(Dense(256, input_shape=noise_shape, activation='relu'))

    #中間層（入力の型は256ノード、出力の型は512ノード、活性化関数はReLU）
    model.add(Dense(512, activation='relu'))

    #中間層（入力の型は512ノード、出力の型は1024ノード、活性化関数はReLU）
    model.add(Dense(1024, activation='relu'))

    #中間層（入力の型は1024ノード、出力の型は784（28×28×1）ノード、活性化関数はtanh）
    model.add(Dense(np.prod(img_shape), activation='tanh'))

    #出力層（28×28×1の画像として出力）
    model.add(Reshape(img_shape))

    model.summary()
    return model

#Dモデルの構築関数
def build_discriminator():
    img_shape = (img_rows, img_cols, channels)
    model = Sequential()

    #入力層（入力の型は画像、出力の型は512ノード、活性化関数はReLU）
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512, activation='relu'))

    #中間層（入力の型は512ノード、出力の型は256ノード、活性化関数はReLU）
    model.add(Dense(256, activation='relu'))

    #出力層（シグモイド関数で0〜1の値として出力）
    model.add(Dense(1, activation='sigmoid'))

    model.summary()
    return model

#GモデルとDモデルを合併させる関数（Dモデルの学習は停止）
def build_combined1():
    discriminator.trainable = False
    model = Sequential([generator, discriminator])
    return model


#入力画像（mnistデータ）の型
img_rows = 28 
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

#潜在変数の次元数 
z_dim = 100

#最適化手法（Adamを使用（学習率、浮動小数点数を引数））
optimizer = Adam(0.0002, 0.5)

#Dモデル構築(欠損関数、最適化手法、評価関数)
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy'])

#Gモデル構築(欠損関数、最適化手法、評価関数)
generator = build_generator()
combined = build_combined1() #Dモデルとのコンビネーションネットワークとして最適化させる
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              P

# 学習部分
- MNISTの画像データ読み込み
- 訓練データの画像と、Gモデルの生成した画像を用意
- Dモデル：訓練データは「１」と認識し、生成データは「０」と認識するように、損失関数を元にそれぞれ学習
- Gモデル：生成データをDモデルが「１」と認識するように、損失関数を元に学習

参考
- [欠損値関数とは？](損失関数を元に学習)

In [3]:
def train(epochs, batch_size=128, save_interval=50): 
    # 教師データ（mnist）の画像を読み込み
    (X_train, _), (_, _) = mnist.load_data()

    # 画像のそれぞれの値を-1〜1に規格化
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)

    half_batch = int(batch_size / 2)

    # batch_size × epochs回繰り返す
    for epoch in range(epochs):
        for iteration in range(batch_size):
            # バッチサイズの半数をGeneratorから生成
            noise = np.random.normal(0, 1, (half_batch, z_dim))
            gen_imgs = generator.predict(noise)

            # バッチサイズの半数を教師データからピックアップ
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            # ---------------------
            #  Discriminatorの学習
            # ---------------------
            # 訓練データを「1」と認識するよう、欠損値関数を元に学習
            d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))

            # 生成データを「0」と認識するよう、欠損値関数を元に学習
            d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))

            # 欠損値の平均を算出
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Generatorの学習
            # ---------------------
            noise = np.random.normal(0, 1, (batch_size, z_dim))
            valid_y = np.array([1] * batch_size)

            # 生成データをDモデルが「１」と認識するように、損失関数を元に学習
            g_loss = combined.train_on_batch(noise, valid_y)

## 生成画像の保存
- 保存先の設定

In [0]:
def save_imgs(epoch):
    # 生成画像を敷き詰めるときの行数、列数
    r, c = 5, 5

    noise = np.random.normal(0, 1, (r * c, z_dim))
    gen_imgs = generator.predict(noise)

    # 生成画像を0-1に再スケール
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("drive/My Drive/GAN_images/mnist_%d.png" % epoch)
    plt.close()

# 実行

In [0]:
import time

t1 = time.time() 
train(epochs=10, batch_size=100, save_interval=1)　#　エポック数、バッチサイズ、画像保存頻度
t2 = time.time()

# 経過時間を表示
elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}")

Number of batches: 100
epoch:0, iter:0,  [D loss: 0.413749, acc.: 72.00%] [G loss: 0.849203]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0, iter:1,  [D loss: 0.333665, acc.: 81.00%] [G loss: 0.925321]
epoch:0, iter:2,  [D loss: 0.311607, acc.: 83.00%] [G loss: 1.020845]
epoch:0, iter:3,  [D loss: 0.247104, acc.: 96.00%] [G loss: 1.134141]
epoch:0, iter:4,  [D loss: 0.231100, acc.: 94.00%] [G loss: 1.207508]
epoch:0, iter:5,  [D loss: 0.216186, acc.: 98.00%] [G loss: 1.294431]
epoch:0, iter:6,  [D loss: 0.188102, acc.: 100.00%] [G loss: 1.429263]
epoch:0, iter:7,  [D loss: 0.172220, acc.: 100.00%] [G loss: 1.525172]
epoch:0, iter:8,  [D loss: 0.161006, acc.: 100.00%] [G loss: 1.555645]
epoch:0, iter:9,  [D loss: 0.147624, acc.: 99.00%] [G loss: 1.641025]
epoch:0, iter:10,  [D loss: 0.125126, acc.: 100.00%] [G loss: 1.751947]
epoch:0, iter:11,  [D loss: 0.119357, acc.: 100.00%] [G loss: 1.789836]
epoch:0, iter:12,  [D loss: 0.105801, acc.: 100.00%] [G loss: 1.845316]
epoch:0, iter:13,  [D loss: 0.104781, acc.: 100.00%] [G loss: 1.968103]
epoch:0, iter:14,  [D loss: 0.098580, acc.: 100.00%] [G loss: 2.080174]
epoch:0